In [102]:
import sys
import os
from bisect import bisect_left
path_to_AOC_helpers = "/Users/connor/Desktop/Coding Problems/Advent_of_code/2024"
# Add the parent directory to the sys.path
sys.path.append(os.path.abspath(path_to_AOC_helpers))
from collections import defaultdict, deque, Counter, namedtuple
from functools import reduce, cache, cmp_to_key
from heapq import heappush, heappop, heapify
from itertools import permutations, combinations, product
from math import gcd, sqrt, factorial, ceil, floor
from sortedcontainers import SortedDict, SortedList, SortedSet

from copy import deepcopy
from math import prod
import re
import numpy as np
import sys
import pyomo.environ as pymo
import pyomo.opt as pyopt
from AOC_helpers import *
sys.setrecursionlimit(10000)

input_file = 'input.txt'
test_file = 'test.txt'

# Get our problem input 
with open(input_file, 'r') as file:
    lines = file.readlines()
    grid = [ row.strip() for row in lines] 
    R, C = len(grid), len(grid[0])

In [103]:
directions = [(0,1), (0, -1), (1, 0), (-1, 0)]
inf = float('inf')
# first, get distance from position on path to end 

def get_endpoints(grid):
    """Returns the start and end points of the race"""
    start = None
    end = None 
    # just loop over grid and update references
    for r in range(R):
        for c in range(C):
            if grid[r][c] == 'S':
                start = (r, c) 
            elif grid[r][c] == 'E':
                end = r, c
    return start, end 

def get_distances(grid, end):
    """Return's the distances for each path on the racetrack
    Do a simple BFS with distance. """
    # 
    distances = defaultdict(lambda : inf)
    q = deque([(end, 0)])
    visited = set()
    while q:
        pos, d = q.popleft()
        if pos in visited:
            continue
        visited.add(pos)
        r, c = pos
        distances[(r, c)] = d
        for dr, dc in directions:
            nr, nc = r + dr, c + dc 
            if (nr, nc) not in visited and grid[nr][nc] != '#' :
                q.append([(nr, nc), d + 1])
    return distances

# ok now, we need to consider all 2D paths of length at most around a given point that save time. 
# Now it can leave the path and come back on and leave again as long is cheat is active and the total length is at most 20.

# so how can we do this? 
# well, for each cheat, we start on the path and end on the path, These are just two points
# so consider all the path pairs that would have a saved distance of 20 or more. Find their manhattan distance. 
# If is is less than 20 then we can cheat to it somehow (otherwise would not be a saving)
# Let's try to make the pairs the lazy way and see if it's fast enough 

def dist(pos1, pos2):
    """Manhattan distance between two points """
    r1, c1 = pos1
    r2, c2 = pos2
    return abs(r1 - r2) + abs(c1 - c2)

def num_cheating_paths(distances, saving_threshold, cheat_duration):
    """Using the distances, return the number of cheating paths of max 
    length cheat_duration that save at least saving_threshold seconds. """
    total_saved = defaultdict(int)
    distance_list = [(d, pos) for pos, d in distances.items()]
    distance_list.sort()
    n = len(distance_list)
    for i in range(n):
        d1, pos1 = distance_list[i]
        j = bisect_left(distance_list, (d1 + saving_threshold - 1, (-1, -1)))
        for k in range(j, n):
            d2, pos2 = distance_list[k]
            if dist(pos1, pos2) > cheat_duration: # too far away to cheat to 
                continue
            saved = (d2 - d1) - dist(pos1, pos2)
            if saved >= saving_threshold:
                total_saved[saved] += 1

    print(f'We found there are {sum(total_saved.values())} new paths
          that save at least {saving_threshold} with max cheat duration {cheat_duration}')
    return sum(total_saved.values())
        

In [104]:
# Solve 
start, end = get_endpoints(grid)
distances = get_distances(grid, end)
num_cheating_paths(distances, 100, 2)
num_cheating_paths(distances, 100, 20)

We found there are 1415 new paths that save at least 100 with max cheat duration 2
We found there are 1022577 new paths that save at least 100 with max cheat duration 20


1022577

In [105]:
# Intial solution to part 1 before adapting solution to part 1. 

# # now we have all the distances from end, we need them in distance  now we need to cheat 
# total_saved  = defaultdict(int)
# threshold = 50
# for r,c in distances.keys():
#     for dr, dc in directions:
#         nr, nc = r + 2 * dr, c + 2 * dc 
#         if (nr, nc) in distances and grid[r + dr][c + dc] == '#':
#             d_1 = distances[(r, c)]
#             d_2 = distances[(nr, nc)]
#             saved = d_1 - d_2 - 2
#             if saved > 0:
#                 total_saved[saved] += 1
# print(f'there are {total_saved} shortcuts that save {threshold} or more seconds')
# print(f'Note that normal path has length of {distances[start]}')